## Домашняя работа №7. Pandas - функции и группировки

### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [5]:
LOW_RATING_NAME = 'low'
AVERAGE_RATING_NAME = 'average'
HIGTH_RATING_NAME = 'hight'

def set_rating(rating: float) -> str:
    if rating <= 2:
        return LOW_RATING_NAME
    
    elif rating <= 4:
        return AVERAGE_RATING_NAME
    
    return HIGTH_RATING_NAME


df = pd.read_csv('ratings.csv')
df['class'] = df.rating.apply(set_rating)
df.head(10)

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,average
1,1,3,4.0,964981247,average
2,1,6,4.0,964982224,average
3,1,47,5.0,964983815,hight
4,1,50,5.0,964982931,hight
5,1,70,3.0,964982400,average
6,1,101,5.0,964980868,hight
7,1,110,4.0,964982176,average
8,1,151,5.0,964984041,hight
9,1,157,5.0,964984100,hight


### Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [15]:
undefined_region_name = 'undefined'

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }

def identify_region(query: str, regions_map: dict) -> str:
    for region_name, cityes_list in regions_map.items():
        if any(city in query for city in cityes_list):
            return region_name

    return undefined_region_name

df = pd.read_csv('keywords.csv')

df['region'] = df.keyword.apply(identify_region, regions_map=geo_data)

df_to_show = df.sort_values(by='region', ascending=False)

In [16]:
df_to_show.head(10)

,keyword,shows,region
6999,москва 24,37459,Центр
61986,такси везет москва официальный сайт,6063,Центр
43404,суперджоб москва вакансии,8208,Центр
73226,фанатик спартак москва,5353,Центр
68005,гостиница москва,5651,Центр
7090,гисметео ярославль,37008,Центр
97382,комсомольская правда москва,3708,Центр
35939,семейный доктор москва официальный сайт,9268,Центр
51916,сбербанк официальный сайт москва,7225,Центр
27302,расписание электричек тверь москва,23272,Центр


In [17]:
df_to_show.tail(10)

,keyword,shows,region
33460,свет,9829,undefined
33459,аниме про демонов,9838,undefined
33458,df,9830,undefined
33457,конус,9830,undefined
33456,др,9830,undefined
33455,тобот мультфильм на русском смотреть онлайн,9790,undefined
33454,логические загадки,10369,undefined
33453,пенсионный фонд рф,9875,undefined
33452,olesya fattakhova,9832,undefined
99999,ггму,3630,undefined


### Задание 3

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    2.1. для каждой строки пройдите по всем годам списка years

    2.2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска

    2.3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
    

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [20]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

years = [str(i) for i in range(1950, 2011)]


def get_production_year(row, years_list):
    for year in years_list:
        if year in row['title']:
            return year

    return 1900


movies['year'] = movies.apply(get_production_year, axis=1, years_list=years)

ratings = ratings.join(other=movies, on='movieId', how='left', rsuffix='_movies')
ratings = ratings[['year', 'rating']].groupby('year').mean().sort_values('rating', ascending=False)

print('Top rated movies production years:')
print(ratings.head(10))

Top rated movies production years:
        rating
year          
1997  3.927552
1970  3.717277
1981  3.716854
2009  3.678468
1966  3.648958
1984  3.613818
1987  3.603044
1965  3.600432
1969  3.591093
1963  3.582237
